In [1]:
look_back = 60
file_modello = "LSTM_prezzo.keras"

In [2]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from joblib import load

pd.set_option("display.max_columns", None)
elenco_usin = pd.read_parquet("Tickers_De_Giro.parquet")
elenco_usin = elenco_usin.loc[elenco_usin["Ticker"] == "BTG", ["USIN", "Ticker"]]
model = load_model(file_modello)
usin = elenco_usin.iloc[0]
print(f"\rScarico dati ticker {usin['Ticker']}                                             ", end=" ", flush=True)
ticker = yf.download(usin['USIN'], start='2019-08-01', end='2023-08-31', progress=False)
print(f"\rCalcolo indicatori ticker {usin['Ticker']}                                             ", end=" ", flush=True)
ticker = fx.crea_indicatori(ticker)
features, target = fx.features_e_target_regressione(ticker)
ticker["Target"] = target
X, Y = fx.converti_in_XY(features, target, look_back)
#X_scaler = load("Scalers/BTG_X_scaler.joblib")
#Y_scaler = load("Scalers/BTG_Y_scaler.joblib")
#n_samples, n_timesteps, n_features = X.shape
#X_scaled = X.reshape((n_samples, n_timesteps * n_features))
#X_scaled = X_scaler.transform(X_scaled)
#X_scaled = X_scaled.reshape((-1, n_timesteps, n_features))
print(f"\rPrevisione {usin['Ticker']}                                             \n", end=" ", flush=True)
pred = model.predict(X)
#pred = Y_scaler.inverse_transform(pred.reshape(-1, 1))


Previsione BTG                                                             
24/24 [==============================] - 2s 18ms/step


In [3]:
ticker = ticker.iloc[look_back:]
ticker["Pred"] = pred
pred

array([[4.194103 ],
       [4.040313 ],
       [4.3077035],
       [4.552556 ],
       [4.637179 ],
       [4.6386805],
       [4.4500194],
       [4.2937016],
       [4.2688785],
       [4.3698406],
       [4.5840573],
       [4.570941 ],
       [4.772576 ],
       [4.6628194],
       [4.7145524],
       [4.8987393],
       [4.980885 ],
       [5.039505 ],
       [5.028992 ],
       [5.058784 ],
       [5.0037184],
       [5.057307 ],
       [5.051985 ],
       [5.1340585],
       [5.1455984],
       [5.162762 ],
       [5.161038 ],
       [5.1548758],
       [5.156522 ],
       [5.136397 ],
       [5.152792 ],
       [5.1625757],
       [5.174911 ],
       [5.181366 ],
       [5.191938 ],
       [5.1799917],
       [5.1652794],
       [5.1656632],
       [5.158878 ],
       [5.172522 ],
       [5.184108 ],
       [5.1867642],
       [5.186794 ],
       [5.1836586],
       [5.1808395],
       [5.1846423],
       [5.148074 ],
       [5.1389236],
       [5.1582465],
       [5.185483 ],


In [4]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp

close = go.Scatter(
    x = ticker.index,
    y = ticker['Close'],
    mode = 'lines',
    line = dict(color='rgba(0, 0, 0, .9)'),
    name = 'Close'
)
previsione = go.Scatter(
    x = ticker.index,
    y = ticker["Pred"],
    mode = 'lines',
    line = dict(color='red'),
    name = 'Previsione'
)
layout = dict(xaxis = dict(autorange=True),
              yaxis = dict(title = 'Close', autorange=True),
              autosize = True,
              margin = go.layout.Margin(
                  l=0,  # Sinistra
                  r=0,  # Destra
                  b=0,  # Basso
                  t=50,  # Alto
                  pad=0  # Padding
              ),
              legend = dict(traceorder = 'normal', bordercolor = 'black')
)

fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.update_layout(layout)
fig.add_trace(close, row=1, col=1)
fig.add_trace(previsione, row=1, col=1)

pyo.plot(fig, filename="test_modello_regressione.html", auto_open=True)

'test_modello_regressione.html'